In [13]:
###Here, I will download the dataset from Kaggle using Kaggle 
# API and perform other operaions if needed.

In [14]:
#First of all, we need to install kaggle module to use the Kaggle API
%pip install kaggle

Note: you may need to restart the kernel to use updated packages.


In [16]:
import kaggle

In [21]:
import kaggle

In [23]:
!chmod 600 $HOME/.kaggle/kaggle.json

As we can see above, there are many error occured during installing kaggle module.

We rectified that by creating folder for kaggle.json file and changed the permissions for the file not to 
accessed by other users in the system. (I am using Ubuntu 24 linux distro)

**Now, Lets try to download the kaggle dataset**

The following command runs on shell(bash) not a python code.

In [2]:
import os
os.chdir("/media/maadunson/New Volume/Projects/orders_data_analysis/")

In [45]:
!kaggle datasets download ankitbansal06/retail-orders -f orders.csv

Dataset URL: https://www.kaggle.com/datasets/ankitbansal06/retail-orders
License(s): CC0-1.0
100%|█████████████████████████████████████████| 200k/200k [00:00<00:00, 210kB/s]
100%|█████████████████████████████████████████| 200k/200k [00:00<00:00, 210kB/s]


The downloaded file will be in Ubunutu's default home folder, I will change to the projects folder which in another disk

**The file was downloaded and moved !!!!**

Let's unzip and move to the next step

In [46]:
import zipfile
zip_act = zipfile.ZipFile('orders.csv.zip')
zip_act.extractall()
zip_act.close()

**Boom!!! The dataset was unzipped as csv file....**

Let's load the data into pandas dataframe for transforming the data before we move on the SQL part for analysis...

In [3]:
import pandas as pd
df = pd.read_csv('orders.csv')


Let's see the dataframe how it looks

In [4]:
df.head(10)

,Order Id,Order Date,Ship Mode,Segment,Country,City,State,Postal Code,Region,Category,Sub Category,Product Id,cost price,List Price,Quantity,Discount Percent
0,1,2023-03-01,Second Class,Consumer,United States,Henderson,Kentucky,42420,South,Furniture,Bookcases,FUR-BO-10001798,240,260,2,2
1,2,2023-08-15,Second Class,Consumer,United States,Henderson,Kentucky,42420,South,Furniture,Chairs,FUR-CH-10000454,600,730,3,3
2,3,2023-01-10,Second Class,Corporate,United States,Los Angeles,California,90036,West,Office Supplies,Labels,OFF-LA-10000240,10,10,2,5
3,4,2022-06-18,Standard Class,Consumer,United States,Fort Lauderdale,Florida,33311,South,Furniture,Tables,FUR-TA-10000577,780,960,5,2
4,5,2022-07-13,Standard Class,Consumer,United States,Fort Lauderdale,Florida,33311,South,Office Supplies,Storage,OFF-ST-10000760,20,20,2,5
5,6,2022-03-13,Not Available,Consumer,United States,Los Angeles,California,90032,West,Furniture,Furnishings,FUR-FU-10001487,50,50,7,3
6,7,2022-12-28,Standard Class,Consumer,United States,Los Angeles,California,90032,West,Office Supplies,Art,OFF-AR-10002833,10,10,4,3
7,8,2022-01-25,Standard Class,Consumer,United States,Los Angeles,California,90032,West,Technology,Phones,TEC-PH-10002275,860,910,6,5
8,9,2023-03-23,Not Available,Consumer,United States,Los Angeles,California,90032,West,Office Supplies,Binders,OFF-BI-10003910,20,20,3,2
9,10,2023-05-16,Standard Class,Consumer,United States,Los Angeles,California,90032,West,Office Supplies,Appliances,OFF-AP-10002892,90,110,5,3


In [5]:
df['Segment'].unique()

array(['Consumer', 'Corporate', 'Home Office'], dtype=object)

First of all, let's change the columns name to lower case separated with underscore.


In [6]:
df.columns = df.columns.str.lower()
df.columns = df.columns.str.replace(' ', '_')

In [7]:
df.head(5)

,order_id,order_date,ship_mode,segment,country,city,state,postal_code,region,category,sub_category,product_id,cost_price,list_price,quantity,discount_percent
0,1,2023-03-01,Second Class,Consumer,United States,Henderson,Kentucky,42420,South,Furniture,Bookcases,FUR-BO-10001798,240,260,2,2
1,2,2023-08-15,Second Class,Consumer,United States,Henderson,Kentucky,42420,South,Furniture,Chairs,FUR-CH-10000454,600,730,3,3
2,3,2023-01-10,Second Class,Corporate,United States,Los Angeles,California,90036,West,Office Supplies,Labels,OFF-LA-10000240,10,10,2,5
3,4,2022-06-18,Standard Class,Consumer,United States,Fort Lauderdale,Florida,33311,South,Furniture,Tables,FUR-TA-10000577,780,960,5,2
4,5,2022-07-13,Standard Class,Consumer,United States,Fort Lauderdale,Florida,33311,South,Office Supplies,Storage,OFF-ST-10000760,20,20,2,5


We can see that the columns names are in lower case sepaarated by underscores

In [17]:
df['ship_mode'].unique()

array(['Second Class', 'Standard Class', 'Not Available', 'unknown',
       'First Class', nan, 'Same Day'], dtype=object)

From the above, we can see that the column ship_mode has values that is related to null values but not Null.

We have to change them to null

In [19]:
df['ship_mode'].replace(['Not Available', 'unknown'], pd.NA, inplace=True)

In [20]:
df['ship_mode'].unique()

array(['Second Class', 'Standard Class', <NA>, 'First Class', nan,
       'Same Day'], dtype=object)

We can see that the values are converted to NaN values. Let's proceed with next transformations.

Finding the discount , sale price and profit columns

In [30]:
df['discount'] = df['list_price'] * df['discount_percent'] * 0.01  #discount
df['sale_price'] = df['list_price'] - df['discount']   #sale price
df['profit'] = df['sale_price'] - df['cost_price'] # profit

In [31]:
df

,order_id,order_date,ship_mode,segment,country,city,state,postal_code,region,category,sub_category,product_id,cost_price,list_price,quantity,discount_percent,discount,sale_price,profit
0,1,2023-03-01,Second Class,Consumer,United States,Henderson,Kentucky,42420,South,Furniture,Bookcases,FUR-BO-10001798,240,260,2,2,5.2,254.8,14.8
1,2,2023-08-15,Second Class,Consumer,United States,Henderson,Kentucky,42420,South,Furniture,Chairs,FUR-CH-10000454,600,730,3,3,21.9,708.1,108.1
2,3,2023-01-10,Second Class,Corporate,United States,Los Angeles,California,90036,West,Office Supplies,Labels,OFF-LA-10000240,10,10,2,5,0.5,9.5,-0.5
3,4,2022-06-18,Standard Class,Consumer,United States,Fort Lauderdale,Florida,33311,South,Furniture,Tables,FUR-TA-10000577,780,960,5,2,19.2,940.8,160.8
4,5,2022-07-13,Standard Class,Consumer,United States,Fort Lauderdale,Florida,33311,South,Office Supplies,Storage,OFF-ST-10000760,20,20,2,5,1.0,19.0,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9989,9990,2023-02-18,Second Class,Consumer,United States,Miami,Florida,33180,South,Furniture,Furnishings,FUR-FU-10001889,30,30,3,4,1.2,28.8,-1.2
9990,9991,2023-03-17,Standard Class,Consumer,United States,Costa Mesa,California,92627,West,Furniture,Furnishings,FUR-FU-10000747,70,90,2,4,3.6,86.4,16.4
9991,9992,2022-08-07,Standard Class,Consumer,United States,Costa Mesa,California,92627,West,Technology,Phones,TEC-PH-10003645,220,260,2,2,5.2,254.8,34.8
9992,9993,2022-11-19,Standard Class,Consumer,United States,Costa Mesa,California,92627,West,Office Supplies,Paper,OFF-PA-10004041,30,30,4,3,0.9,29.1,-0.9
